In [6]:
import json
import os
import boto3
import rasterio
import pandas as pd

# Create metadata output file

In [4]:
out_dir = os.getcwd()
metadata_file = os.path.join(out_dir, 'metadata\\metadata_treecover_tml.json')

# Load input data

In [7]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv') 
boundary_georef = boundary_georef.dropna(subset=['units_boundary_name']).reset_index(drop=True)
print(boundary_georef.to_string())

             geo_name   level aoi_boundary_name units_boundary_name           city_name  country_name country_code continent
0         ARG-Mendoza  region         ADM3union                ADM3             Mendoza     Argentina          ARG   America
1   ARG-Mar_del_Plata    city              ADM3                ADM4  Mar del Plata city     Argentina          ARG   America
2         ARG-Ushuaia    city              ADM4                ADM5        Ushuaia city     Argentina          ARG   America
3           ARG-Salta  region         ADM2union                ADM3               Salta     Argentina          ARG   America
4    ARG-Buenos_Aires  region         ADM2union                ADM2        Buenos Aires     Argentina          ARG   America
5        BRA-Teresina    city         ADM4union                ADM4       Teresina city        Brazil          BRA   America
6        BRA-Teresina  region         ADM2union                ADM2     Teresina region        Brazil          BRA   America


In [ ]:
boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
geo_name = boundary_georef.loc[i, 'geo_name']
city_name = boundary_georef.loc[i, 'city_name']
country_name = boundary_georef.loc[i, 'country_name']
continent_name = boundary_georef.loc[i, 'continent']

print(boundary_id)

# read dataset
#dataset = rasterio.open('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/land_use/esa_world_cover/v_0/'+boundary_id+'-ESA-world_cover-2000.tif')

# fill metadata fields
dataset_title = "Trees in Mosaic Landscapes extract for " + city_name
dataset_year = '2020'
dataset_city_name = city_name
dataset_country_name = country_name
dataset_description = "The Trees in Mosaic Landscapes dataset provides tree extent data at 10m scale based on trained Convolutional Neural Network using satellite imagery (Sentinel-1 and Sentinel-2). It enables accurate reporting of tree cover outside of dense, closed-canopy forests, including in urban areas."
dataset_snippet = "Tree cover data for "+ city_name + ', ' + country_name + ' ('+ dataset_year + ')' 
dataset_tags = [geo_name,
                "Greenspace",
                "Biodiversity",
                "Tree cover",
                "Geography: " + continent_name,
                "Geography: " + continent_name + ": " + country_name,
                "Geography: " + continent_name + ": " + country_name + ": "+ city_name,
                "Time: "+ dataset_year]
dataset_spatial_resolution = '10m'
dataset_temporal_resolution = 'yearly'
dataset_spatial_extent = 'Global'
dataset_temporal_extent = '2020'
dataset_extent = [[dataset.bounds[0],dataset.bounds[1]],[dataset.bounds[2],dataset.bounds[3]]]
dataset_format = 'raster'
dataset_data_source = 'Trees in Mosaic Landscapes'
dataset_source_url = 'https://www.landcarbonlab.org/data/#trees-in-mosaic-landscapes'
dataset_provider = 'World Resources Institute (WRI)'
dataset_url = "https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/land_use/esa_world_cover/v_0/"+boundary_id+"-ESA-world_cover-2000.tif"
dataset_status = 'published'
dataset_license = 'CC BY 4.0'
dataset_crs = 'ESPG:4326'